In [1]:
%%capture

%cd ..


In [2]:
from datasets import load_dataset
import numpy as np

from similarity_augmentations import consts

mmlu = load_dataset(consts.MMLU_HF_PATH, name="all")
print(mmlu)

mmlu_train_ctx = mmlu["auxiliary_train"]["question"]
print(len(mmlu_train_ctx))
print(mmlu["auxiliary_train"][0]["question"])
print(mmlu_train_ctx[0])


Found cached dataset mmlu (/home/mgallo/.cache/huggingface/datasets/cais___mmlu/all/1.0.0/49beb454d1ef6d7994f43183dc194713f39dd672b4f5a92c08b0b394991910c7)


  0%|          | 0/4 [00:00<?, ?it/s]

DatasetDict({
    auxiliary_train: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 99842
    })
    test: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 14042
    })
    validation: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 1531
    })
    dev: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 285
    })
})
99842
Davis decided to kill Adams. He set out for Adams's house. Before he got there he saw Brooks, who resembled Adams. Thinking that Brooks was Adams, Davis shot at Brooks. The shot missed Brooks but wounded Case, who was some distance away. Davis had not seen Case. In a prosecution under a statute that proscribes any attempt to commit murder, the district attorney should indicate that the intended victim(s) was/were
Davis decided to kill Adams. He set out for Adams's house. Before he got there he saw B

In [3]:
mutual = load_dataset(consts.MUTUAL_HF_PATH, name="mutual_plus")
print(mutual)

mutualplus_train_ctx = mutual["train"]["article"]
print(len(mutualplus_train_ctx))
print(mutualplus_train_ctx[0])


Found cached dataset mutual_harness (/home/mgallo/.cache/huggingface/datasets/lighteval___mutual_harness/mutual_plus/0.0.1/be31c67b35f0d5c4cce450a4e8475f0eb26ae243f5bfd701a524305282b0873a)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['answers', 'options', 'article', 'id'],
        num_rows: 7088
    })
    test: Dataset({
        features: ['answers', 'options', 'article', 'id'],
        num_rows: 886
    })
    validation: Dataset({
        features: ['answers', 'options', 'article', 'id'],
        num_rows: 886
    })
})
7088
M: Excuse me. How much is this suit? F: It's on sale today for $750. It's normally $900. M: Wow, that is pretty expensive! I was thinking that it might be 4 or 500. F: This material is imported from Italy. It's the finest in the world, and if you bought a suit made of this material at many department stores, you would pay about $2000. M: Uh-hah. But isn't that the point of coming to a market like this, to get a discount compared to the expensive department stores? Besides I saw a suit just like this one a few stalls down, and they were selling it for $600. I still thought that it was too expensive.


**NOTE** Pay attention, in MuTual some `article`s are the same! They differ in the `options` they offer, but we do not 
encode this or consider it for similarity search :/

In [4]:
mutual["train"]["article"][10:13]


["M: Good evening, what can I do for you? F: Good evening. I would like to buy a bottle of perfume. M: OK. We have many kinds of perfume here. You can choose freely. F: Which brand would you recommend? I don't want to buy one with quality problems. M: Don't worry, we have no such problems. How about this one? It sells very well. F: Umm, it smells good. How much is it? M: 400 yuan per bottle. And if you buy 2 bottles at the same time, you can get a 10% discount. We have other special offers as well. If you buy lipstick or face cream, you can get a free gift. F: Thanks. I just need perfume when I want to buy lipstick I will come here again.",
 "M: Good evening, what can I do for you? F: Good evening. I would like to buy a bottle of perfume. M: OK. We have many kinds of perfume here. You can choose freely. F: Which brand would you recommend? I don't want to buy one with quality problems. M: Don't worry, we have no such problems. How about this one? It sells very well. F: Umm, it smells go

In [5]:
mutual["train"]["options"][10:13]


[['M: Sure. It was 400 yuan each bottle early, but now, it is 360 for one. Thank you.',
  'M: Sure. It will be 400 yuan for one bottle. Thank you.',
  'M: I’m sorry, I didn’t understand. Could you repeat a little louder, please?',
  'M: Sure. It will be 440 yuan for one bottle. Thank you.'],
 ['M: OK. Just one bottle of perfume, right?',
  'M: I’m sorry, I didn’t understand. Could you repeat a little louder, please?',
  'M: Sure. One bottle of perfume and a lipstick. It will be 400 yuan in total.',
  'M: Sure. One bottle of face cream, right?'],
 ['M: Sure. Thanks for coming to our store. Here are your perfume and lipstick.',
  'M: Sure. Thanks for coming to our store.',
  'M: I didn’t hear you. Please could you tell me again?',
  'M: Please finish your work on time.']]

In [6]:
from similarity_augmentations import conf
from similarity_augmentations.embedding import faiss_utils

mutualplus_index = faiss_utils.create_or_load_faiss_index(
    conf.VECTORDB_DIR,
    "mutual_plus__train__all-distilroberta-v1",
    dataset=mutualplus_train_ctx,
)
print(mutualplus_index)
mutualplus_index, mutualplus_index.ntotal


[2023-10-10 02:56:44,645: INFO] Loading FAISS index '/gpfs/home5/mgallo/MuTual/vector_db/mutual_plus__train__all-distilroberta-v1.faiss' (similarity_augmentations.embedding.faiss_utils:28)
<faiss.swigfaiss_avx2.IndexFlat; proxy of <Swig Object of type 'faiss::IndexFlat *' at 0x149e29e7dad0> >


(<faiss.swigfaiss_avx2.IndexFlat; proxy of <Swig Object of type 'faiss::IndexFlat *' at 0x149e29e7dad0> >,
 7088)

In [7]:
mmlu_index = faiss_utils.create_or_load_faiss_index(
    conf.VECTORDB_DIR,
    "mmlu__auxiliary_train__all-distilroberta-v1",
    dataset=mmlu_train_ctx,
)
print(mmlu_index)
mmlu_index, mmlu_index.ntotal


[2023-10-10 02:56:44,825: INFO] Loading FAISS index '/gpfs/home5/mgallo/MuTual/vector_db/mmlu__auxiliary_train__all-distilroberta-v1.faiss' (similarity_augmentations.embedding.faiss_utils:28)


<faiss.swigfaiss_avx2.IndexFlat; proxy of <Swig Object of type 'faiss::IndexFlat *' at 0x149e29e7ecd0> >


(<faiss.swigfaiss_avx2.IndexFlat; proxy of <Swig Object of type 'faiss::IndexFlat *' at 0x149e29e7ecd0> >,
 99842)

In [8]:
query_idx = np.random.randint(0, len(mutualplus_train_ctx) + 1)
# query_idx = 2514, 1966
print(query_idx, mutualplus_train_ctx[query_idx])
print()

D, I = mmlu_index.search(mutualplus_index.reconstruct(query_idx)[None, :], 4)
for d, i in zip(D.squeeze(), I.squeeze()):
    print(d, mmlu_train_ctx[i])


5085 M: What's the matter with you, Mary? You looked tired. F: I didn't sleep well last night. M: Maybe a stomachache or headache? F: Neither, the noise almost drove me mad.

0.97763383 Paul was dealing with _ . The new girl Linda in class really liked him a lot. Paul was not happy about it. Paul's good friend, Sam, tried to comfort him.  "What's the big deal? I think Linda is kind of cute and nice enough." Paul glared at Sam. "Then why don't you ask her to be your girlfriend? There she is, sitting all by herself at the other end of the cafeteria. Go there and ask her to be your honey!" "She's crazy about you, not me," Sam replied. "Well, I don't want any girlfriend, especially not a girl like her! "Paul growled. Sam wouldn't give up.  "The only thing pesky about her is that she keeps writing you those notes ." "I don't like notes at all!" Paul interrupted. He took another sip of milk. "Did you notice those strange brown eyes? Why aren't they normal red or pink?" Sam took another look 

**NOTE** 2 things:
1. There are many very similar articles in MMLU as well.
2. MMLU also contain dialogues, and if these are often retrieved by similarity, removing speaker 
   tags doesn't really affect results much...

## Search performance: batched vs single query vs GPU batchd (query `MuTual` key on `MMLU`)

In [10]:
%%timeit

# downside here is that it's difficult to control which points are retrieved, can't simply increase K if that datapoint
# was already retrieved. would need to perform a full ranking over the whole query dataset, then get the kth
# (globally unique) datapoint with a reduction over the retrieved matrix starting from rank 1 / going sequentially like
# in the above loop
choices = np.random.choice(len(mutualplus_train_ctx), 2000, replace=False).tolist()
mmlu_index.search(mutualplus_index.reconstruct_batch(choices), 1)


160 ms ± 10.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [40]:
%%time

full_ranking = mmlu_index.search(
    mutualplus_index.reconstruct_batch(range(mutualplus_index.ntotal)),
    mutualplus_index.ntotal
)
# BLAZING FAST compared to LagChain (has no batching), and it's on CPU


CPU times: user 33.2 s, sys: 1.18 s, total: 34.4 s
Wall time: 10.2 s


Many query datapoints share the same most similar counterparts in MMLU, need a way to work around this.

In [41]:
np.unique(full_ranking[1][:100, 0]).shape


(54,)

In [11]:
import faiss
import torch

if torch.cuda.is_available():
    print("going to GPU")
    res = faiss.StandardGpuResources()
    print(res)
    mmlu_index = faiss.index_cpu_to_gpu(res, 0, mmlu_index)
    mmlu_index


going to GPU
<faiss.swigfaiss_avx2.StandardGpuResources; proxy of <Swig Object of type 'faiss::gpu::StandardGpuResources *' at 0x149d43113a20> >


In [44]:
%%time

try:
    full_ranking_gpu = mmlu_index.search(
        mutualplus_index.reconstruct_batch(range(mutualplus_index.ntotal)),
        mutualplus_index.ntotal,
    )
except Exception as e:
    print(e)


Error in void faiss::gpu::validateKSelect(int) at /home/conda/feedstock_root/build_artifacts/faiss-split_1685210641191/work/faiss/gpu/impl/IndexUtils.cu:26: Error: 'k > 0 && k <= getMaxKSelection()' failed: GPU index only supports min/max-K selection up to 2048 (requested 7088)
CPU times: user 524 ms, sys: 13.4 ms, total: 537 ms
Wall time: 32.1 ms


In [45]:
%%timeit

choices = np.random.choice(len(mutualplus_train_ctx), 2000, replace=False).tolist()
mmlu_index.search(mutualplus_index.reconstruct_batch(choices), 1)


18.8 ms ± 406 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Almost a 10X speedud between CPU and GPU.

Since not all MuTual datapoints can be searched in the same batch, searching has to be sub-batched in sizes of 2048.

In [13]:
from pathlib import Path

from similarity_augmentations.embedding import faiss_utils

offline_path = Path(conf.ROOT_DIR / "euclidean_sim_search_k_5.npz")

if offline_path.is_file():
    storage = np.load(offline_path)
    D_mmlu, I_mmlu = storage["D"], storage["I"]
else:
    D_mmlu, I_mmlu = faiss_utils.kth_similar_to_all_query_vectors(
        mutualplus_index, mmlu_index, 5
    )
    np.savez(offline_path, D=D_mmlu, I=I_mmlu)

print(D_mmlu.shape, I_mmlu.shape)


(7088, 5) (7088, 5)


In [12]:
I_mmlu[:10], D_mmlu[:10]


(array([[42541, 76198, 42540, 76197, 82668],
        [79730, 79727, 79728, 79729, 88818],
        [73972, 67549, 85755, 85356, 79863],
        [77756, 77758, 77757, 59443, 77308],
        [79019, 79020, 79021, 80113, 77756],
        [71768, 79020, 79019, 79018, 79021],
        [71766, 71768, 91834, 91832, 71765],
        [88821, 79727, 88818, 79728, 88822],
        [77962, 77961, 77960, 77959, 77963],
        [72130, 72129, 72127, 72126, 73972]]),
 array([[0.96991038, 0.97956085, 0.97980195, 1.01031911, 1.01647377],
        [0.87837648, 0.90300608, 0.91198033, 0.91227961, 0.91522765],
        [0.80798781, 0.85060573, 0.8509689 , 0.85247672, 0.85260928],
        [0.81840825, 0.83044231, 0.8401593 , 0.85221905, 0.85763788],
        [0.86363435, 0.86955786, 0.87006474, 0.87723446, 0.87857223],
        [0.99462008, 1.00072527, 1.00403428, 1.02579594, 1.02616894],
        [0.71662378, 0.71825588, 0.77716231, 0.79258281, 0.81728649],
        [0.86571741, 0.92043138, 0.9220016 , 0.94135708, 0

In [52]:
(2 - D_mmlu) / 2


array([[0.51504481, 0.51021957, 0.51009902, 0.49484044, 0.49176311],
       [0.56081176, 0.54849696, 0.54400983, 0.5438602 , 0.54238617],
       [0.5960061 , 0.57469714, 0.57451555, 0.57376164, 0.57369536],
       ...,
       [0.57952267, 0.57952267, 0.57952267, 0.57952267, 0.56398505],
       [0.62264222, 0.61529797, 0.61072779, 0.61072779, 0.61072779],
       [0.58851922, 0.57111576, 0.56800672, 0.56219983, 0.56194547]])

In [54]:
uniques_and_counts = [np.unique(x, return_counts=True) for x in I_mmlu.T]


In [55]:
for idx, (u, _) in enumerate(uniques_and_counts, start=1):
    print(
        f"rank {idx} unique datapoints {len(u)} prop {len(u)/len(mutualplus_train_ctx):.3f}"
    )
print(sum(u.shape[0] for u, _ in uniques_and_counts))
uniques_and_counts


rank 1 unique datapoints 2176 prop 0.307
rank 2 unique datapoints 2481 prop 0.350
rank 3 unique datapoints 2628 prop 0.371
rank 4 unique datapoints 2692 prop 0.380
rank 5 unique datapoints 2856 prop 0.403
12833


[(array([  200,   254,   581, ..., 98806, 99379, 99747]),
  array([1, 2, 2, ..., 1, 1, 3])),
 (array([  199,   581,   593, ..., 97853, 98034, 98759]),
  array([1, 1, 1, ..., 7, 1, 3])),
 (array([  144,   201,   416, ..., 98759, 99179, 99278]),
  array([1, 1, 1, ..., 1, 1, 1])),
 (array([  144,   428,   593, ..., 97547, 97853, 98154]),
  array([1, 2, 1, ..., 1, 8, 1])),
 (array([  417,   721,   797, ..., 98759, 99456, 99747]),
  array([1, 1, 1, ..., 1, 1, 1]))]

In [26]:
from itertools import chain

from typing import Dict, List, Union


def mean_distance_at_rank(
    I: np.ndarray,
    D: np.ndarray,
    rank: int,
    prioritize: str = "size",
    higher_is_better: bool = False,
) -> List[Dict[str, Union[int, List[int], float]]]:
    rank -= 1
    assert I.shape == D.shape and 0 <= rank <= I.shape[1]
    mean_distance_dicts = []
    for u in np.unique(I[:, rank]):
        distance_idxs = np.argwhere(I[:, rank] == u)
        mean_distance_dicts.append(
            {
                "id": u,
                "distance": D[distance_idxs, rank].mean(),
                "similar_to": list(chain(*distance_idxs.tolist())),
            }
        )
    return sort_similarities(mean_distance_dicts, prioritize, higher_is_better)


def sort_similarities(
    distance_at_rank: List[Dict],
    prioritize: str,
    higher_is_better: bool,
) -> List[Dict]:
    assert prioritize in ["size", "distance"]
    sort_key, reverse = lambda d: len(d["similar_to"]), True
    if prioritize == "distance":
        sort_key = lambda d: d["distance"]
        if not higher_is_better:
            reverse = False
    return list(sorted(distance_at_rank, key=sort_key, reverse=reverse))


In [27]:
x = mean_distance_at_rank(I_mmlu, D_mmlu, 1)
print(len(x))
x


2176


[{'id': 79019,
  'distance': 0.9474879771294679,
  'similar_to': [4,
   29,
   30,
   63,
   66,
   81,
   84,
   95,
   107,
   108,
   122,
   125,
   130,
   165,
   186,
   195,
   209,
   249,
   254,
   265,
   266,
   268,
   280,
   281,
   282,
   284,
   285,
   298,
   327,
   328,
   369,
   371,
   399,
   537,
   543,
   544,
   553,
   554,
   555,
   558,
   560,
   613,
   622,
   634,
   659,
   660,
   710,
   711,
   713,
   714,
   715,
   716,
   720,
   722,
   732,
   743,
   760,
   763,
   764,
   765,
   769,
   770,
   786,
   787,
   793,
   813,
   817,
   834,
   879,
   911,
   913,
   914,
   915,
   916,
   917,
   918,
   922,
   933,
   947,
   1041,
   1043,
   1044,
   1046,
   1127,
   1144,
   1145,
   1147,
   1156,
   1179,
   1190,
   1201,
   1250,
   1253,
   1258,
   1314,
   1316,
   1323,
   1324,
   1349,
   1372,
   1381,
   1383,
   1390,
   1398,
   1403,
   1421,
   1422,
   1436,
   1437,
   1518,
   1533,
   1541,
   1542,
   1543,

In [34]:
from functools import reduce


def ranked_distance_merge(
    I: np.ndarray,
    D: np.ndarray,
    prioritize: str = "size",
    higher_is_better: bool = False,
) -> List[Dict]:
    def merge_fn(acc: Dict, distance_d: List[Dict]) -> Dict:
        for dp_d in distance_d:
            id = dp_d["id"]
            distance = dp_d["distance"]
            similar_to = dp_d["similar_to"]
            if id in acc:
                acc[id]["distance"] = (acc[id]["distance"] + distance) / 2
                acc[id]["similar_to"] += similar_to
                acc[id]["similar_to"] = list(set(acc[id]["similar_to"]))
            else:
                acc[id] = {"distance": distance, "similar_to": similar_to}
        return acc

    mean_distances_at = [
        mean_distance_at_rank(I, D, k, prioritize, higher_is_better)
        for k in range(1, I.shape[1] + 1)
    ]
    merged = reduce(merge_fn, mean_distances_at, {})
    merged = [{"id": id, **d} for id, d in merged.items()]
    return sort_similarities(merged, prioritize, higher_is_better)


In [35]:
y = ranked_distance_merge(I_mmlu, D_mmlu)
print(len(y))
y


7502


[{'id': 79019,
  'distance': 0.9850332678821335,
  'similar_to': [4,
   5,
   14,
   20,
   21,
   23,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   62,
   63,
   64,
   65,
   66,
   81,
   82,
   83,
   84,
   95,
   97,
   106,
   107,
   108,
   113,
   122,
   123,
   125,
   130,
   131,
   133,
   150,
   153,
   158,
   159,
   165,
   167,
   169,
   170,
   176,
   185,
   186,
   189,
   195,
   196,
   209,
   212,
   228,
   248,
   249,
   250,
   253,
   254,
   255,
   257,
   260,
   265,
   266,
   268,
   272,
   273,
   274,
   279,
   280,
   281,
   282,
   283,
   284,
   285,
   296,
   297,
   298,
   305,
   317,
   326,
   327,
   328,
   355,
   356,
   367,
   368,
   369,
   371,
   379,
   388,
   390,
   397,
   398,
   399,
   403,
   404,
   410,
   419,
   420,
   450,
   451,
   460,
   469,
   470,
   500,
   502,
   520,
   521,
   537,
   543,
   544,
   553,
   554,
   555,
   558,
   559,
   560,
   581,
   596,
   597,
   602,
   603,
   

In [37]:
for i, d in enumerate(y):
    print(i, d["id"], d["distance"] ,len(d["similar_to"]))
    if len(d["similar_to"]) <= 10:
        break


0 79019 0.9850332678821335 1897
1 79020 0.967609593137225 1706
2 79018 0.971114901147436 1463
3 79021 0.9540578996332783 1269
4 73972 0.9233526392964012 1151
5 79022 0.9538448876350153 1025
6 73971 0.9236228116503267 979
7 69437 1.0040792207881617 187
8 67639 0.8918517899208822 163
9 50068 1.0024635113690827 150
10 76571 0.9114604262770521 135
11 67637 0.8983545040029195 123
12 69436 1.0024082323048846 120
13 67638 0.8701752174375714 113
14 79730 0.8426566658017001 108
15 12188 0.939424144579541 102
16 79728 0.8675338171904292 101
17 50066 0.9344990251770948 99
18 50069 1.0434149103577084 98
19 69030 1.073735021660306 96
20 91533 1.0056801248676772 94
21 72129 0.9885820808651957 86
22 76572 0.9625348209452758 79
23 91534 1.0306308709651406 76
24 12187 0.9317811220897166 75
25 89710 0.9948782627829613 74
26 91536 0.9475788358569589 73
27 12186 0.9575867797820804 70
28 74725 1.0734945926637876 69
29 79727 0.8904905430370509 69
30 60305 1.0009008857046628 67
31 45955 1.1280431262084416 65

In [ ]:
# TODO check percentiles, do some plotting


In [ ]:
from similarity_augmentations.finetuning import augmentations

p = 0.5
scaled_p = len(mutualplus_train_ctx) * p / len(mmlu_train_ctx)
print(p, round(scaled_p, 3))
n_samples = augmentations.to_mmlu_size(scaled_p, len(mmlu_train_ctx))
n_samples


0.5 0.035


3544

In [ ]:
from tqdm.autonotebook import tqdm

sampled = set()
closest_mmlu_points = {}
closest_dict = {}

for i, mutual_dp in tqdm(enumerate(mutualplus_train_ctx), total=len(mutualplus_train_ctx)):
    found, idx = False, 1
    while not found and idx <= mmlu_index.index.ntotal:
        mmlu_closest = mmlu_index.similarity_search_with_score_by_vector(
            mutualplus_index.index.reconstruct(i).tolist(), k=idx
        )
        kth_closest_doc, kth_distance = mmlu_closest[idx - 1]
        kth_closest_mmlu_idx = kth_closest_doc.metadata["idx"]
        if i not in closest_dict:
            closest_dict[i] = kth_closest_mmlu_idx
        if kth_closest_mmlu_idx not in sampled:
            sampled.add(kth_closest_mmlu_idx)
            closest_mmlu_points[i] = {
                "idx": kth_closest_mmlu_idx,
                "rank": idx,
                "distance": kth_distance,
            }
            found = True
        else:
            idx += 1


0it [00:00, ?it/s]

KeyboardInterrupt: 

In [ ]:
closest_mmlu_points


{0: {'idx': 42541, 'rank': 1, 'distance': 0.9699097},
 1: {'idx': 79730, 'rank': 1, 'distance': 0.8783765},
 2: {'idx': 73972, 'rank': 1, 'distance': 0.80798745},
 3: {'idx': 77756, 'rank': 1, 'distance': 0.81840813},
 4: {'idx': 79019, 'rank': 1, 'distance': 0.86363363},
 5: {'idx': 71768, 'rank': 1, 'distance': 0.9946202},
 6: {'idx': 71766, 'rank': 1, 'distance': 0.7166224},
 7: {'idx': 88821, 'rank': 1, 'distance': 0.8657171},
 8: {'idx': 77962, 'rank': 1, 'distance': 0.73718405},
 9: {'idx': 72130, 'rank': 1, 'distance': 0.8492526},
 10: {'idx': 82902, 'rank': 1, 'distance': 0.7559252},
 11: {'idx': 82900, 'rank': 2, 'distance': 0.7593404},
 12: {'idx': 82899, 'rank': 3, 'distance': 0.7646302},
 13: {'idx': 89993, 'rank': 1, 'distance': 1.0443623},
 14: {'idx': 79020, 'rank': 1, 'distance': 1.0587243},
 15: {'idx': 48032, 'rank': 1, 'distance': 0.9345333},
 16: {'idx': 48034, 'rank': 2, 'distance': 0.9732434},
 17: {'idx': 87426, 'rank': 1, 'distance': 0.87675095},
 18: {'idx': 87

In [ ]:
dict(sorted(closest_mmlu_points.items(), key=lambda v: v[1]["distance"]))


{862: {'idx': 87178, 'rank': 1, 'distance': 0.39217412},
 863: {'idx': 79752, 'rank': 3, 'distance': 0.43426543},
 861: {'idx': 87380, 'rank': 1, 'distance': 0.4701337},
 1621: {'idx': 13347, 'rank': 1, 'distance': 0.48573753},
 800: {'idx': 47989, 'rank': 3, 'distance': 0.5116639},
 799: {'idx': 47991, 'rank': 2, 'distance': 0.520386},
 60: {'idx': 79865, 'rank': 1, 'distance': 0.5209559},
 61: {'idx': 79863, 'rank': 2, 'distance': 0.5212731},
 798: {'idx': 47990, 'rank': 1, 'distance': 0.53459895},
 1385: {'idx': 68732, 'rank': 2, 'distance': 0.5475934},
 202: {'idx': 50411, 'rank': 2, 'distance': 0.5498006},
 1843: {'idx': 31863, 'rank': 1, 'distance': 0.5540807},
 1741: {'idx': 91347, 'rank': 1, 'distance': 0.56371224},
 203: {'idx': 50412, 'rank': 3, 'distance': 0.5652811},
 1844: {'idx': 31864, 'rank': 2, 'distance': 0.567515},
 1790: {'idx': 20656, 'rank': 2, 'distance': 0.5709613},
 571: {'idx': 53132, 'rank': 2, 'distance': 0.57630384},
 1254: {'idx': 57326, 'rank': 1, 'distan